# 事前準備

## ライブラリ

In [155]:
import urllib.parse
import urllib.request
import json
import pandas as pd
import numpy as np

## estat-API

APIキーの設定

In [1]:
ESTAT_APPID = '724e5b90772a3e9289f41a253e4e7e32438f4fff'

estat-APIからレスポンスを取得する関数

In [3]:
def get_estat_response(params):
  p = params.copy()

  # appId
  p['appId'] = ESTAT_APPID
  
  # url生成
  url = 'http://api.e-stat.go.jp/rest/2.1/app/json/getStatsData?'
  url += urllib.parse.urlencode(p)

  with urllib.request.urlopen(url) as response:
    return json.loads(response.read().decode('utf-8'))

レスポンスのJSONをDataFrameに整形する関数

In [144]:
def conv_estat_response_to_dataframe(response):
  # CLASS_INF
  CLASS_OBJ = response['GET_STATS_DATA']['STATISTICAL_DATA']['CLASS_INF']['CLASS_OBJ']

  # TABLE_INF
  TABLE_INF = response['GET_STATS_DATA']['STATISTICAL_DATA']['TABLE_INF']

  # VALUE
  VALUE = response['GET_STATS_DATA']['STATISTICAL_DATA']['DATA_INF']['VALUE']

  # VALUEをDataFrameに変換
  df_res = pd.json_normalize(VALUE)

  # CLASS_OBJのDataFrameを結合
  for d in CLASS_OBJ :
    # DataFrameに変換 '@code','@name'だけ抽出
    df_class = pd.json_normalize(d['CLASS']) 
    df_class = df_class.copy()[['@code','@name']]

    # @codeをキー名に変更 @nameをキー名＋'_name'に変更
    key_name = '@{}'.format(d['@id'])
    df_class = df_class.rename(columns={'@code':key_name,'@name': key_name+'_name'})

    # DataFrameを結合
    df_res = pd.merge(df_res, df_class, on=key_name, how='outer')

  # 統計情報を追加
  df_res['statsDataId'] = TABLE_INF['@id']
  df_res['statsDataName'] = TABLE_INF['STAT_NAME']['$']
  
  return df_res

# 港湾統計

## 2010年～2017年

[estat-API](https://www.e-stat.go.jp/dbview?sid=0003130803)からデータを取得する

In [127]:
params = {
    'statsDataId' : '0003130803',
    'cdCat02' : '100',
    'cdCat03' : '100',
}

res = get_estat_response(params)
print(res)

{'GET_STATS_DATA': {'RESULT': {'STATUS': 0, 'ERROR_MSG': '正常に終了しました。', 'DATE': '2022-09-22T09:54:41.207+09:00'}, 'PARAMETER': {'LANG': 'J', 'STATS_DATA_ID': '0003130803', 'NARROWING_COND': {'CODE_CAT02_SELECT': 100, 'CODE_CAT03_SELECT': 100}, 'DATA_FORMAT': 'J', 'START_POSITION': 1, 'METAGET_FLG': 'Y'}, 'STATISTICAL_DATA': {'RESULT_INF': {'TOTAL_NUMBER': 4510, 'FROM_NUMBER': 1, 'TO_NUMBER': 4510}, 'TABLE_INF': {'@id': '0003130803', 'STAT_NAME': {'@code': '00600280', '$': '港湾調査'}, 'GOV_ORG': {'@code': '00600', '$': '国土交通省'}, 'STATISTICS_NAME': '港湾統計（年報）', 'TITLE': {'@no': '2-3-1', '$': '第２部\u3000甲種港湾 第３表\u3000海上出入貨物表\u3000（１）トン数総数表 甲種港湾 2010年～'}, 'CYCLE': '年次', 'SURVEY_DATE': 0, 'OPEN_DATE': '2018-02-06', 'SMALL_AREA': 0, 'MAIN_CATEGORY': {'@code': '10', '$': '運輸・観光'}, 'SUB_CATEGORY': {'@code': '01', '$': '運輸'}, 'OVERALL_TOTAL_NUMBER': 17380, 'UPDATED_DATE': '2022-07-29', 'STATISTICS_NAME_SPEC': {'TABULATION_CATEGORY': '港湾統計（年報）'}, 'TITLE_SPEC': {'TABLE_CATEGORY': '第２部\u3000甲種港湾', 'TABL

レスポンスのJSONをDataFrameに整形

In [205]:
df_res = conv_estat_response_to_dataframe(res)
df_res

,@tab,@cat01,@cat02,@cat03,@cat04,@time,@unit,$,@tab_name,@cat01_name,@cat02_name,@cat03_name,@cat04_name,@time_name,statsDataId,statsDataName
0,120,100,100,100,01000,2010000000,トン,190367576,トン数,合計,計,計,北海道,2010年,0003130803,港湾調査
1,120,110,100,100,01000,2010000000,トン,3641962,トン数,輸出,計,計,北海道,2010年,0003130803,港湾調査
2,120,120,100,100,01000,2010000000,トン,36090854,トン数,輸入,計,計,北海道,2010年,0003130803,港湾調査
3,120,130,100,100,01000,2010000000,トン,75494601,トン数,移出,計,計,北海道,2010年,0003130803,港湾調査
4,120,140,100,100,01000,2010000000,トン,75140159,トン数,移入,計,計,北海道,2010年,0003130803,港湾調査
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4588,NaN,NaN,NaN,NaN,46002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,鹿児島県_加治木,NaN,0003130803,港湾調査
4589,NaN,NaN,NaN,NaN,46004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,鹿児島県_喜入,NaN,0003130803,港湾調査
4590,NaN,NaN,NaN,NaN,46005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,鹿児島県_川内,NaN,0003130803,港湾調査
4591,NaN,NaN,NaN,NaN,46006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,鹿児島県_米之津,NaN,0003130803,港湾調査


In [206]:
# 必要な列だけ抽出
df_res =  df_res[['statsDataName','@cat01_name','@time_name','$','@unit','@cat04','@cat04_name']]

# 列名の変更
columns = {'@cat01_name':'種類','@time_name':'年次','@cat04':'港湾コード','@cat04_name':'港湾名','$':'取扱貨物量','@unit':'単位'}
df_res = df_res.rename(columns=columns)

# 欠損データ削除
df_res = df_res.dropna()

df_res

,statsDataName,種類,年次,取扱貨物量,単位,港湾コード,港湾名
0,港湾調査,合計,2010年,190367576,トン,01000,北海道
1,港湾調査,輸出,2010年,3641962,トン,01000,北海道
2,港湾調査,輸入,2010年,36090854,トン,01000,北海道
3,港湾調査,移出,2010年,75494601,トン,01000,北海道
4,港湾調査,移入,2010年,75140159,トン,01000,北海道
...,...,...,...,...,...,...,...
4505,港湾調査,合計,2017年,1051802,トン,47007,沖縄県_石垣
4506,港湾調査,輸出,2017年,2815,トン,47007,沖縄県_石垣
4507,港湾調査,輸入,2017年,40176,トン,47007,沖縄県_石垣
4508,港湾調査,移出,2017年,352868,トン,47007,沖縄県_石垣


In [230]:
df_pivot = df_res.copy()

# 取扱貨物量の欠損値を処理して数値に変換
df_pivot = df_pivot.replace('-', '0')
df_pivot = df_pivot.astype({'取扱貨物量': 'int'})

# クロス集計
df_pivot = pd.pivot_table(df_pivot, index=['港湾名','種類'], columns='年次',values='取扱貨物量',aggfunc=np.sum).reset_index()

df_pivot

年次,港湾名,種類,2010年,2011年,2012年,2013年,2014年,2015年,2016年,2017年
0,三重県,合計,NaN,NaN,64796525.0,62510340.0,63941182.0,NaN,NaN,NaN
1,三重県,移入,NaN,NaN,5770420.0,6431855.0,6695726.0,NaN,NaN,NaN
2,三重県,移出,NaN,NaN,15559712.0,13920666.0,15373861.0,NaN,NaN,NaN
3,三重県,輸入,NaN,NaN,39912016.0,38120375.0,38013776.0,NaN,NaN,NaN
4,三重県,輸出,NaN,NaN,3554377.0,4037444.0,3857819.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
600,鹿児島県_鹿児島,合計,40220057.0,39797738.0,39400674.0,39827495.0,38722283.0,34703113.0,34006495.0,34738544.0
601,鹿児島県_鹿児島,移入,20564919.0,20455814.0,20448680.0,20763653.0,20133788.0,17728699.0,17207308.0,17850130.0
602,鹿児島県_鹿児島,移出,18273515.0,17893985.0,17431160.0,17850846.0,17430003.0,15408797.0,15661639.0,15577484.0
603,鹿児島県_鹿児島,輸入,1373825.0,1447939.0,1509987.0,1208336.0,1153781.0,1550591.0,1101319.0,1292767.0


In [157]:
# CSVに書き出し
df_pivot.to_csv('全国港湾の取扱貨物量（合計）.csv',encoding='cp932')

## 2020年

[国土交通省ホームページ](https://www.mlit.go.jp/k-toukei/kowannenpodb.html)のEXCELデータから作業する

例えば2020年の場合はhttps://www.mlit.go.jp/k-toukei/content/001448034.xlsx

In [212]:
# ファイル名の指定
input_file_name = 'https://www.mlit.go.jp/k-toukei/content/001448034.xlsx'
input_book = pd.ExcelFile(input_file_name)

# 統計結果は2番目のシート、上から2行は無視
input_sheet_name = input_book.sheet_names
df_org = input_book.parse(input_sheet_name[1],skiprows = 2)

df_org

,都道府県,Unnamed: 1,港格,港湾,Unnamed: 4,種別,Unnamed: 6,合計,輸出,輸入,移出,移入
0,北海道,1,地方港湾,合計,NaN,一般,1.0,91743748,3045266.0,26759896.0,30127021.0,31811565.0
1,北海道,1,地方港湾,合計,NaN,自航,2.0,90269445,NaN,NaN,44205590.0,46063855.0
2,北海道,1,重要港湾,稚内,1002.0,一般,1.0,361697,33.0,1803.0,75040.0,284821.0
3,北海道,1,重要港湾,稚内,1002.0,自航,2.0,1073930,NaN,NaN,547890.0,526040.0
4,北海道,1,重要港湾,紋別,1048.0,一般,1.0,299245,1452.0,101281.0,71521.0,124991.0
...,...,...,...,...,...,...,...,...,...,...,...,...
320,沖縄県,47,重要港湾,運天,47002.0,自航,2.0,569520,NaN,NaN,285215.0,284305.0
321,沖縄県,47,重要港湾,平良,47003.0,一般,1.0,893011,3688.0,17360.0,129278.0,742685.0
322,沖縄県,47,重要港湾,平良,47003.0,自航,2.0,23275,NaN,NaN,20160.0,3115.0
323,沖縄県,47,重要港湾,石垣,47004.0,一般,1.0,726181,1963.0,17217.0,131313.0,575688.0


2010年～2017年と同型式になるよう、整形する

In [235]:
def format_dataframe(df_arg,time):
  df = df_arg.copy()

  # 都道府県と港湾で集計
  df = df.groupby(['都道府県','港湾']).agg({'合計': 'sum','輸出': 'sum','輸入': 'sum','移出': 'sum','移入': 'sum'})
  
  # 列の値を行にstack
  df = pd.DataFrame(df.stack()).reset_index()

  # 列名の整形
  df['港湾名'] = df.apply(lambda x: '{}_{}'.format(x['都道府県'],x['港湾']) if x['港湾'] != '合計' else x['都道府県'], 1)
  df[time] = df.apply(lambda x: x[0], 1)
  df['種類'] = df.apply(lambda x: x['level_2'], 1)

  # 必要な列だけ抽出
  df = df[['港湾名','種類',time]]
  
  return df

In [236]:
time = '2020年'

df_2020 = format_dataframe(df_org,time)
df_2020

,港湾名,種類,2020年
0,三重県,合計,59851445.0
1,三重県,輸出,3750166.0
2,三重県,輸入,32932508.0
3,三重県,移出,16240813.0
4,三重県,移入,6927958.0
...,...,...,...
1020,鹿児島県_鹿児島,合計,27746958.0
1021,鹿児島県_鹿児島,輸出,1599.0
1022,鹿児島県_鹿児島,輸入,1232606.0
1023,鹿児島県_鹿児島,移出,12309449.0


2010年～2017年と結合

In [ ]:
df_last = pd.merge(df_pivot, df_2020, on=['港湾名','種類'] , how='left')
df_last

## 2018年・2019年

In [ ]:
# ファイル名の指定
input_file_name = 'https://www.mlit.go.jp/k-toukei/content/001448034.xlsx'
input_book = pd.ExcelFile(input_file_name)

# 統計結果は2番目のシート、上から2行は無視
input_sheet_name = input_book.sheet_names
df_org = input_book.parse(input_sheet_name[1],skiprows = 2)

df_org